# train

In [ ]:
# Case1: 
rest_len = 14
eis_len = 107
directory_path = r"data_120s\train"

# Case2: 
rest_len = 120  # 120: 60min / 60: 30min / 20: 10min
eis_len = 36
directory_path = r"data_K\train"


## Transformer

In [ ]:
import torch
from models import MultiFeatureTransformer
from train import train_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_dir = "model"
model = MultiFeatureTransformer(
    input_dim = 4,
    output_dim = 2,
    seq_len_in = rest_len,
    seq_len_out = eis_len-1,
    d_model = 256,
    nhead = 4,
    num_layers = 6
    )

train_model(model, directory_path, device, model_dir, "Transformer", rest_len, eis_len, epochs=10)


## BiLSTM

In [ ]:
import torch
from models import ImprovedBiLSTMModel
from train import train_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ================ BiLSTM ================
model_dir = "model_BiLSTM"
bilstm_model = ImprovedBiLSTMModel(
    input_dim=4, 
    output_dim=2, 
    hidden_dim=128, 
    seq_len_in=rest_len,
    seq_len_out=eis_len-1,
    num_layers=3,
    dropout=0.3
    )

train_model(bilstm_model, directory_path, device, model_dir, "BiLSTM", rest_len, eis_len, epochs=1000)

In [ ]:
import torch
from models import GCNSeq2Seq
from train import train_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ================ GCN ================
model_dir = "model_GCN"
gcn_model = GCNSeq2Seq(
    in_features=4,
    hidden_dim=64,
    out_features=2,
    input_len=rest_len,
    output_len=eis_len-1
)

edge_index = torch.tensor([[0, 0, 1, 1, 2, 2, 3, 3],
                           [1, 2, 0, 3, 0, 1, 1, 2]], dtype=torch.long)

# edge_index = torch.tensor([
#     [0, 0, 1, 1, 1, 2, 2, 3, 3, 3],
#     [1, 3, 0, 2, 3, 1, 3, 0, 1, 2]
# ], dtype=torch.long)

train_model(gcn_model, directory_path, device, model_dir, "GCN", rest_len, eis_len, epochs=1000, lr=1e-3, edge_index=edge_index)


# test

In [ ]:
# Case1: 
data_path = r"data_120s\test"

# Case2:
data_path = r"data_K\test"


## Transformer

In [ ]:
import time
import os
from test_my import extract_number_rest,extract_number_eis,test_result_get_single,extract_info,save_single_eis_results_as_pkl
from models import ImprovedBiLSTMModel

model_path = "model"
result_dir = "result"
model = MultiFeatureTransformer(
    input_dim = 4,
    output_dim = 2,
    seq_len_in = rest_len,
    seq_len_out = eis_len-1,
    d_model = 256,
    nhead = 4,
    num_layers = 6
    )

start_time = time.time()

for cell in os.listdir(data_path):
    print(cell)
    cell_path = os.path.join(data_path, cell)

    if os.path.isdir(cell_path):
        print(f"Running test for cell: {cell}")

        cell_path = rf"{data_path}\{cell}"
        
        input_files = [f for f in os.listdir(cell_path) if f.endswith(".pkl") and 'rest_' in f]
        sorted_input_files = sorted(input_files, key=lambda x: extract_number_rest(x))

        output_files = [f for f in os.listdir(cell_path) if f.endswith(".pkl") and 'eis_' in f]
        sorted_output_files = sorted(output_files, key=lambda x: extract_number_eis(x))

        for input_file, output_file in zip(sorted_input_files,sorted_output_files):
            test_preds = test_result_get_single(model_path, cell_path, input_file, model)

            eis_number, cap_value = extract_info(output_file)
            result_file = rf"{result_dir}\{cell}\predicted_eis_{eis_number}_cap_{cap_value}.pkl"

            save_single_eis_results_as_pkl(rest_len, test_preds, result_file, cell_path, output_file)

end_time = time.time()
test_duration = end_time - start_time
print(f"Time: {test_duration:.2f} seconds")


## BiLSTM

In [ ]:
import time
import os
from test_my import extract_number_rest,extract_number_eis,test_result_get_single,extract_info,save_single_eis_results_as_pkl
from models import ImprovedBiLSTMModel

# ================ BiLSTM ================
model_path = "model_BiLSTM"
result_dir = "result_BiLSTM"
model = ImprovedBiLSTMModel(
    input_dim=4, 
    output_dim=2, 
    hidden_dim=128, 
    seq_len_in=rest_len,
    seq_len_out=eis_len-1,
    num_layers=3,
    dropout=0.3
    )

start_time = time.time()

for cell in os.listdir(data_path):
    print(cell)
    cell_path = os.path.join(data_path, cell)

    if os.path.isdir(cell_path):
        print(f"Running test for cell: {cell}")

        cell_path = rf"{data_path}\{cell}"
        
        input_files = [f for f in os.listdir(cell_path) if f.endswith(".pkl") and 'rest_' in f]
        sorted_input_files = sorted(input_files, key=lambda x: extract_number_rest(x))

        output_files = [f for f in os.listdir(cell_path) if f.endswith(".pkl") and 'eis_' in f]
        sorted_output_files = sorted(output_files, key=lambda x: extract_number_eis(x))

        for input_file, output_file in zip(sorted_input_files,sorted_output_files):
            test_preds = test_result_get_single(model_path, cell_path, input_file, model)

            eis_number, cap_value = extract_info(output_file)
            result_file = rf"{result_dir}\{cell}\predicted_eis_{eis_number}_cap_{cap_value}.pkl"

            save_single_eis_results_as_pkl(rest_len, test_preds, result_file, cell_path, output_file)

end_time = time.time()
test_duration = end_time - start_time
print(f"Time: {test_duration:.2f} seconds")


## GCN

In [ ]:
import time
import os
import torch
from test_my import extract_number_rest,extract_number_eis,test_result_get_single,extract_info,save_single_eis_results_as_pkl
from models import GCNSeq2Seq

# ================ GCN ================
model_path = "model_GCN"
result_dir = "result_GCN"
model = GCNSeq2Seq(
    in_features=4,
    hidden_dim=64,
    out_features=2,
    input_len=rest_len,
    output_len=eis_len-1
)

edge_index = torch.tensor([
    [0, 0, 1, 1, 1, 2, 2, 3, 3, 3],
    [1, 3, 0, 2, 3, 1, 3, 0, 1, 2]
], dtype=torch.long)

start_time = time.time()  # 记录开始时间

for cell in os.listdir(data_path):
    print(cell)
    cell_path = os.path.join(data_path, cell)

    if os.path.isdir(cell_path):
        print(f"Running test for cell: {cell}")

        cell_path = rf"{data_path}\{cell}"
        
        # 获取文件夹中的文件列表
        input_files = [f for f in os.listdir(cell_path) if f.endswith(".pkl") and 'rest_' in f]
        sorted_input_files = sorted(input_files, key=lambda x: extract_number_rest(x))

        output_files = [f for f in os.listdir(cell_path) if f.endswith(".pkl") and 'eis_' in f]
        sorted_output_files = sorted(output_files, key=lambda x: extract_number_eis(x))

        for input_file, output_file in zip(sorted_input_files,sorted_output_files):
            test_preds = test_result_get_single(model_path, cell_path, input_file, model, edge_index)

            eis_number, cap_value = extract_info(output_file)
            result_file = rf"{result_dir}\{cell}\predicted_eis_{eis_number}_cap_{cap_value}.pkl"

            save_single_eis_results_as_pkl(test_preds, result_file, cell_path, output_file)

end_time = time.time()
test_duration = end_time - start_time
print(f"Time: {test_duration:.2f} seconds")


# Result

In [ ]:
from test_my import error_excel_get
import os

result_path = "result"
# result_path = "result_GCN"
# result_path = "result_BiLSTM"

cell_folders = [f for f in os.listdir(result_path) if os.path.isdir(os.path.join(result_path, f))]

for cell in cell_folders:
    cell_path = os.path.join(result_path,cell)
    save_path = os.path.join(result_path, f"{cell}.xlsx")
    error_excel_get(cell_path, save_path, cell)
